## Preliminary Operations


In [1]:
import requests

In [2]:
# 1) go to https://developer.spotify.com/console/get-audio-analysis-track/?id=06AKEBrKUckW0KREUWRnvT
# 2) press "try it"
# 3) login
# 4) agree 
# 5) execute this cell and give the script the token (see above)
if "token" not in locals(): # if you have not inserted the token 
    token=input("Give me the token")

header={"Authorization": "Bearer %s"%token}


In [3]:
# %% Search api: first info

params={"q": "Staying alive bee gees", "type": "track"}
search_url="https://api.spotify.com/v1/search"
req=requests.get(url=search_url, params=params,headers=header)
assert req.status_code==200, req.content
answer=req.json()
items=answer["tracks"]["items"]
first_result=items[0]
print("First result")
print("Author: %s"%first_result["artists"][0]["name"])
print("Name: %s"%first_result["name"])
print("Preview url: %s"%first_result["preview_url"])
print("Id on spotify: %s"%first_result["id"])

First result
Author: Bee Gees
Name: Stayin Alive
Preview url: https://p.scdn.co/mp3-preview/f7a8a8c787b2a8ba31e78c42fa791663c4af396d?cid=774b29d4f13844c495f206cafdad9c86
Id on spotify: 5ubvP9oKmxLUVq506fgLhk


In [4]:
# %% Audio feature APIs

modes=["minor", "major"]
key_tonal=["C","C#", "D","D#","E","F","F#","G","G#","A","A#","B"]

audio_feature_url="https://api.spotify.com/v1/audio-features"
params={"ids":first_result["id"]}
req=requests.get(url=audio_feature_url, params=params, headers=header)
audio_features=req.json()["audio_features"][0]
#print(audio_features)
print("Duration: %.3f seconds"%(audio_features["duration_ms"]/1000))
print("BPM: %d"%audio_features["tempo"])
print("Key: %s-%s"%(key_tonal[audio_features["key"]], 
                   modes[audio_features["mode"]]))

for feature in ["danceability", "energy", "speechiness", "acousticness","liveness","instrumentalness","valence"]:
    print("The %s of the song is %1.f %%"%(feature, 100*audio_features[feature]))


Duration: 285.373 seconds
BPM: 103
Key: A#-minor
The danceability of the song is 70 %
The energy of the song is 77 %
The speechiness of the song is 3 %
The acousticness of the song is 3 %
The liveness of the song is 15 %
The instrumentalness of the song is 1 %
The valence of the song is 95 %


In [ ]:
audio_features

## Playlist


In [5]:
import os
import json
import time
import requests
import urllib
import numpy as np
os.chdir(os.path.abspath(os.path.dirname("myplaylist")))

audio_feature_url="https://api.spotify.com/v1/audio-features"
add_item_playlist_url="https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
create_playlist_url="https://api.spotify.com/v1/users/{user_id}/playlists"

In [6]:
# %% Get the token
# 1) go to https://developer.spotify.com/console/post-playlists/
# 2) press "get token"
# 3) remember to include playlist-modify-public 
# 4) login
# 5) agree 
# 6) execute this cell and give the script the token (see above)
if "token" not in locals(): # if you have not inserted the token 
    token=input("Please, give me your token\n")
header={"Authorization": "Bearer %s"%token}


In [7]:
# %% Get the list of songs

assert os.path.exists("list_of_songs_dance.json"), "Please put here a list of songs"
with open("list_of_songs_dance.json",'r') as fp:
    ids=json.load(fp)["ids"]

In [ ]:
ids

In [8]:
# %% Get the audio features
audio_features=[]
for id_ in ids:
    params={"ids":id_}
    req=requests.get(url=audio_feature_url, params=params, headers=header)
    assert req.status_code==200, req.content
    audio_features_song=req.json()["audio_features"][0]
    audio_features.append(audio_features_song)
    time.sleep(1) # wait 1 second between the questions

In [ ]:
audio_features[0]

In [9]:
from sklearn import preprocessing

In [6]:
x = np.array([3, 1, 2, 4, 5, 6, 7, 10, 9])
y = np.argsort(x)
y_test = y[:int(.3*len(y))]
y_test2 = y[int(.3*len(y)):]

y_final = np.concatenate((y_test, y_test2))

if np.array_equal(y_final, y):
    print("bella")

bella


In [19]:
x_scaled = preprocessing.minmax_scale(x)
x_scaled

array([0.22222222, 0.        , 0.11111111, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 1.        , 0.88888889])

In [10]:
from random import seed
from random import randint
seed(1)

In [11]:
def random_sort(idx, num_songs):
    count = 1
    for id in range(len(idx)):
        if count == num_songs:
            random_seed = randint(id, len(idx)-1)
            idx[[id,random_seed]] = idx[[random_seed, id]]
            count = 1
            num_songs += 1
        count += 1
    return idx


In [11]:
x_scaled = preprocessing.minmax_scale(x)
y_scaled = preprocessing.minmax_scale(y)
print(x_scaled)
print(y_scaled)
print(np.multiply(x_scaled,y_scaled))
print(np.add(x_scaled, y_scaled))

[0.22222222 0.         0.11111111 0.33333333 0.44444444 0.55555556
 0.66666667 1.         0.88888889]
[0.125 0.25  0.    0.375 0.5   0.625 0.75  1.    0.875]
[0.02777778 0.         0.         0.125      0.22222222 0.34722222
 0.5        1.         0.77777778]
[0.34722222 0.25       0.11111111 0.70833333 0.94444444 1.18055556
 1.41666667 2.         1.76388889]


In [ ]:
print(random_sort(y,4))

In [12]:
def sort_songs(audio_features, danceb = False):
    """"Receive audio features and sort them according to your criterion"

    Args:
        audio_features (list of dictionaries): List of songs with audio features

    Returns:
        list of dict: the sorted list
    """
    sorted_songs=[]
    energy = np.array([af["energy"] for af in audio_features])
    energy = preprocessing.minmax_scale(energy)
    dance = np.array([af["danceability"] for af in audio_features])
    dance = preprocessing.minmax_scale(dance)
    ifeatures = np.add(0.2*energy, 0.8*dance)
    ifeatures = preprocessing.minmax_scale(ifeatures)

    if not danceb:       
        
        idx = np.argsort(energy)
        idx_first = idx[:int(.3*len(idx))] # first 30%
        idx_rest = idx[int(.3*len(idx)):] # rest of the 70%
        idx_rest = random_sort(idx_rest, int(.1*len(idx_rest)))
        idx_new = np.concatenate((idx_first, idx_rest))
        # energy = energy[idx]
    else:
        idx_new = np.argsort(ifeatures)
    
    for idxs in idx_new:
        sorted_songs.append(audio_features[idxs])
        
    return sorted_songs

In [13]:
# %% Now let's create some way to organize them!
dance = True
shuffled_songs=sort_songs(audio_features, dance)

In [14]:
# %% Create the playlist
# Go to https://open.spotify.com/ , top right corner, press "Account"
# look at your username or user_id
name_playlist=input("What's the name of the playlist you want to create?\n")
user_id=input("What's your username?\n")

params={"name":name_playlist, "description": "JAYSOOON TATUUUM!"}

In [15]:
req=requests.post(url=create_playlist_url.format(user_id=user_id), 
                    json=params, headers=header)
assert req.status_code==201, req.content
playlist_info=req.json()
print("Playlist created with url %s"%playlist_info["external_urls"]["spotify"])

Playlist created with url https://open.spotify.com/playlist/4Ue7VjuiDgtW5Vuryj55ZR


In [16]:
# %% Populating the playlist
# Doc at https://developer.spotify.com/documentation/web-api/reference/playlists/add-tracks-to-playlist/
uris=[]
for song in shuffled_songs:
    uris.append(song["uri"])
params={"uris":uris, }
req=requests.post(url=add_item_playlist_url.format(playlist_id=playlist_info["id"]), json=params, headers=header)
assert req.status_code==201, req.content
playlist_info_songs=req.json()